# Intial notebook for exploring and understanding the KBs API:s

Findings: bundles of jp2 and XML (alto OCR containing text) files are collected in "dark-packages". For newspapers one dark package coresponds to one daily issue. A "dataset" contains a number of such dark-packages. So we need to 1) find the correct datset 2) extract all of the dark-packages 3) inspect dark packages to find the actual file-url:s to download.

General findings:
- At some point during exploration the KBs SSL certificate seemed to expire. All code and downloads still work but only by explictly turning off SSL verificaiton for requests.
- Strangely some of the larger datsets contain the arbitrary number 10 000 dark-packages. Seems like some mistake was made by KB and that all data could be made available has not been made available.

#### GET known dark-package with all files (Norrköpings W)

Use these links to fins API queries and info.

https://data.kb.se/docs#/default/list_packages__get
https://data.kb.se/docs#/default/get_datasets_datasets_get
https://data.kb.se/search/docs#/default/query__get


TO GET ALL DATASETS: curl -X 'GET' \
  'https://data.kb.se/search/?q=%2A&%40type=dataset&genreForm=Tidningsnummer&limit=20&offset=0' \
  -H 'accept: application/json'

```bash
curl -X 'GET' \
  'https://data.kb.se/dark-8428195' \
  -H 'accept: application/json'

```
#### GET Aftonbladet whole dataset
```bash
curl -X 'GET' \
  'https://data.kb.se/dataset/7mf87n2g5xzjf8r4' \
  -H 'accept: application/json'

```

This gets the metadata post for the dataset. But it is unusable, as it doesn't contains any links to dark-packages which are the api-objects that contain actual links to image files. Instead, see below.

#### Norrköpings Weckobladh

NOTE: The pages in the dataset browser are not sorted chronologically.

First page url, offset=0: https://data.kb.se/dataset/m1tnm4cpktxgf28n?offset=0

Final page url, offset=9990: https://data.kb.se/dataset/m1tnm4cpktxgf28n?offset=9990

Number of dark-packages: 9990+10(?)

NOTE: Seems like a very strange coincidence that the total number of dark-packages should be 10000 exactly. 1 dark-package is one number/issue.

#### Aftonbladet

First page url, offset=0: https://data.kb.se/dataset/7mf87n2g5xzjf8r4?offset=0

Final page url, offset=9990: https://data.kb.se/dataset/7mf87n2g5xzjf8r4?offset=9990

NOTE: Yep. Aftonbladet cuts off at 10000 too. That is not a coincidence, but arbitrary. The whole datasets have not been made available.


#### Get metadata containing 10000 (arbitrary maximum) dark-package objects

```bash
curl -X 'GET' \
  'https://data.kb.se/search/?q=%2A&%40type=package&inDataset=https%3A%2F%2Fdata.kb.se%2Fdataset%2F7mf87n2g5xzjf8r4&searchGranularity=package&_sort=title&limit=10000&offset=0' \
  -H 'accept: application/json'
```

This gets "all" 10000 dark-packages in one json-object. Of course this is not the whole dataset though...
The response actually contains "total": 10000 which confirms that KB has put in a 10k hardlimit for queries. Unbelievable. The API/web interface was completely unusable in practice before, but this makes it truly unusable in theory as well.



In [1]:
import requests
import json

In [2]:
#from requests.structures import CaseInsensitiveDict

headers = {'accept': 'application/json'}
url = "https://data.kb.se/dark-8428195"
# This gets a dataset, but it is useless as the dataset contains no open references to dark-packages. These probably need to be scraped, incredibly...
#url = "https://data.kb.se/dataset/7mf87n2g5xzjf8r4"
response = requests.get(url, headers=headers)

print(response.text)

# packageid='dark-8428195'
# header = 'accept: application/json'
# x = requests.get('https://data.kb.se/dataset/dark-8428195.jsonld')

# print(x.text)

{"@context":"https://id.kb.se/context.jsonld","@id":"https://data.kb.se/dark-8428195","@type":"FilePackage","created":"2017-11-24T12:00:06.355000+01:00","version":"urn:nbn:se:kb:dark-package-instance-7755896","modified":null,"name":"NORRKÖPINGS TIDNINGAR 1846-06-17","license":null,"inDataset":[{"@id":"https://data.kb.se/dataset/m1tnm4cpktxgf28n","@type":"Dataset","title":"Norrköpings tidningar 1787-1895"}],"relatedTo":{"@id":"https://libris.kb.se/g0s7kqks0mj1ghp#it"},"includes":[{"@id":"https://data.kb.se/dark-8428195/bib15650837_18460617_146391_48_0001.jp2","@type":"File","fileName":"bib15650837_18460617_146391_48_0001.jp2","encodingFormat":{"@id":"https://id.kb.se/encodingFormat/image/jp2","@type":"EncodingFormat","code":"image/jp2"},"describedBy":{"@id":"https://data.kb.se/dark-8428195"},"contentSize":8555569,"checksum":{"@type":"MD5","value":"26673a0ce0359cf99e4102a2b4436b6c"},"height":4969,"width":3438},{"@id":"https://data.kb.se/dark-8428195/bib15650837_18460617_146391_48_0002.jp

In [3]:
json_response = json.loads(response.text)

print(type(response))
print(type(json_response))

# for key, value in json_response.items():
#     print(key, ' : ', value)

<class 'requests.models.Response'>
<class 'dict'>


In [4]:
#print(type(json_response['includes']))
l = json_response['includes']
print("Length: "+str(len(l)))

fileurl = l[0]['@id']
filename = l[0]['fileName']
md5 = l[0]['checksum']['value']
print(f'------ /CDHU/ -------')
print(f'fileurl: {fileurl}')
print(f'filename: {filename}')
print(f'md5: {md5}')

for i in range(len(l)):
    fileurl = l[i]['@id']
    filename = l[i]['fileName']
    md5 = l[i]['checksum']['value']
    print(f'------ /CDHU/ -------')
    print(f'fileurl: {fileurl}')
    print(f'filename: {filename}')
    print(f'md5: {md5}')

#for key, value in json_response['includes'].items():
#    print(key, ' : ', value)

Length: 12
------ /CDHU/ -------
fileurl: https://data.kb.se/dark-8428195/bib15650837_18460617_146391_48_0001.jp2
filename: bib15650837_18460617_146391_48_0001.jp2
md5: 26673a0ce0359cf99e4102a2b4436b6c
------ /CDHU/ -------
fileurl: https://data.kb.se/dark-8428195/bib15650837_18460617_146391_48_0001.jp2
filename: bib15650837_18460617_146391_48_0001.jp2
md5: 26673a0ce0359cf99e4102a2b4436b6c
------ /CDHU/ -------
fileurl: https://data.kb.se/dark-8428195/bib15650837_18460617_146391_48_0002.jp2
filename: bib15650837_18460617_146391_48_0002.jp2
md5: 9fb99a4d051318c0dabcfcdb7d63d583
------ /CDHU/ -------
fileurl: https://data.kb.se/dark-8428195/bib15650837_18460617_146391_48_0003.jp2
filename: bib15650837_18460617_146391_48_0003.jp2
md5: 76be6b8230c72ec5d18bdf61193f6c8c
------ /CDHU/ -------
fileurl: https://data.kb.se/dark-8428195/bib15650837_18460617_146391_48_0004.jp2
filename: bib15650837_18460617_146391_48_0004.jp2
md5: 88a25c91bb1e83fe0ef445c3e47be92f
------ /CDHU/ -------
fileurl: htt

In [ ]:
# SAVED JSON DATA from requests:

In [ ]:
{
    "@id": "https://data.kb.se/search?q=%2A&%40type=dataset&genreForm=Tidningsnummer&limit=20&offset=0",
    "total": 23,
    "hits": [
        {
            "@context": "https://id.kb.se/context.jsonld",
            "@id": "https://data.kb.se/dataset/hwjb1l34fgzl1202",
            "@type": "Dataset",
            "title": [
                "Kistebrev (1700-1800-tal)"
            ],
            "description": [
                "Dessa kistebrev ingår i Kungl. bibliotekets samling av Kistebrev, vilken innehåller cirka 900 kistebrev och cirka 200 i nytryck. Kistebrev är enkla handkolorerade ettbladstryck, främst träsnitt, som ofta klistrades in i kistlock, ursprungligen möjligen i skyddande, senare enbart i dekorativt syfte. Kistebreven trycktes under 1700-talet och fram till mitten av 1800-talet i Sverige bland annat i Lund, Växjö, Jönköping, Karlskrona, Karlshamn och Varberg. Motiven var till större delen bibliska och moraliska, men även historiska och folkloristiska motiv förekom, liksom porträtt av kungar och drottningar."
            ],
            "keyword": [
                "bibliotek",
                "forskning",
                "historia",
                "kulturarv"
            ],
            "alternative": null
        },
        {
            "@context": "https://id.kb.se/context.jsonld",
            "@id": "https://data.kb.se/dataset/jxkmbknkg2p8mgld",
            "@type": "Dataset",
            "title": [
                "Kartor"
            ],
            "description": [
                "Omfattar ett antal digitaliserade kartor från KB:s samlingar samt en digitaliserad version av \"Descriptionis Ptolemaicæ avgmentvm siue Occidentis notitia breui commentario illustrata"
            ],
            "keyword": [
                "bibliotek",
                "forskning",
                "historia",
                "kulturarv"
            ],
            "alternative": null
        },
        {
            "@context": "https://id.kb.se/context.jsonld",
            "@id": "https://data.kb.se/dataset/3h55fqlg12mbr980",
            "@type": "Dataset",
            "title": [
                "Handskrifter"
            ],
            "description": [
                "Omfattar \"Fornsvenska handskrifter\" samt verken \"Codex Aureus\"  (A 135), \"Ballades et poèmes diverses\" av François Villon (Vu 22) och \"Resedagbok 1687-1688\" av Nicodemus Tessin (Signum: KB HS Acc. 2001/88). -- \"Fornsvenska handskrifter\" innehåller KB:s fornsvenska handskrifter. Mappstrukturen innehåller handskriftens signum -- \"Resedagbok 1687-1688\" innehåller text på tyska, italienska och franska."
            ],
            "keyword": [
                "bibliotek",
                "forskning",
                "historia",
                "kulturarv"
            ],
            "alternative": null
        },
        {
            "@context": "https://id.kb.se/context.jsonld",
            "@id": "https://data.kb.se/dataset/crcrp2nz979j3ws5",
            "@type": "Dataset",
            "title": [
                "Tryckta böcker"
            ],
            "description": [
                "Omfattar \"Svenskt tryck före 1700\" samt verken  \"Gesta danorum\" av Saxo Grammaticus och \"Descriptionis Ptolemaicæ avgmentvm siue Occidentis notitia breui commentario illustrata\".  -- \"Svenskt tryck före 1700\" var ett pilotprojekt som hade flera syften. Det övergripande syftet var att söka lämpliga metoder för beskrivning och digitalisering av äldre material ur Kungl. bibliotekets samlingar. Materialet tillgängliggjordes via Kungl. bibliotekets webb 2002. Detta dataset innehåller bildfilerna till respektive verk. --  Digitaliserad version av förstautgåvan av Saxo Grammaticus verk Gesta danorum (”Danernas bedrifter”) tryckt i Paris 1514. Detta exemplar innehåller omfattande marginalanteckningar på latin av bröderna Olaus och Johannes Magnus, samt Olaus Magnus handskrivna ögonvittnesskildring av Stockholms blodbad 1520. Signum: KB HS D 461b. -- Digitaliserad version av \"Descriptionis Ptolemaicæ avgmentvm siue Occidentis notitia breui commentario illustrata"
            ],
            "keyword": [
                "bibliotek",
                "forskning",
                "historia",
                "kulturarv"
            ],
            "alternative": null
        },
        {
            "@context": "https://id.kb.se/context.jsonld",
            "@id": "https://data.kb.se/dataset/7mf75j765jqnqcjq",
            "@type": "Dataset",
            "title": [
                "Svenska språkmodeller"
            ],
            "description": [
                "Språkmodeller för svenska, bl.a SpaCY och transformer-modeller såsom BERT."
            ],
            "keyword": [
                "bibliotek",
                "forskning",
                "historia",
                "kulturarv"
            ],
            "alternative": null
        },
        {
            "@context": "https://id.kb.se/context.jsonld",
            "@id": "https://data.kb.se/dataset/m1tnm4cpktxgf28n",
            "@type": "Dataset",
            "title": [
                "Norrköpings tidningar 1787-1895"
            ],
            "description": [
                "Digitaliserad version av Kungliga bibliotekets bestånd av Norrköpings Tidningar 1787-1895. Digitaliseringen genomfördes 2017-2019. Specifikationer: https://www.kb.se/namespace/digark/deliveryspecification/agreement/dig_tidn/index.html"
            ],
            "keyword": [
                "bibliotek",
                "forskning",
                "historia",
                "kulturarv"
            ],
            "alternative": null
        },
        {
            "@context": "https://id.kb.se/context.jsonld",
            "@id": "https://data.kb.se/dataset/6ld76n1p44s12ht1",
            "@type": "Dataset",
            "title": [
                "Dagens nyheter"
            ],
            "description": [
                "Digitaliserad version av Dagens Nyheter. Digitaliseringen är gjord från KB:s pappersförlagor och genomfördes 2013-2014. Specifikationer: https://www.kb.se/namespace/digark/deliveryspecification/agreement/dig_tidn/index.html"
            ],
            "keyword": [
                "bibliotek",
                "forskning",
                "historia",
                "kulturarv"
            ],
            "alternative": null
        },
        {
            "@context": "https://id.kb.se/context.jsonld",
            "@id": "https://data.kb.se/dataset/7mf87n2g5xzjf8r4",
            "@type": "Dataset",
            "title": [
                "Aftonbladet"
            ],
            "description": [
                "Digitaliserad version av Aftonbladet 1831-1900. Digitaliseringen är gjord från KB:s pappersförlagor och genomfördes 2013-2014. Specifikationer: https://www.kb.se/namespace/digark/deliveryspecification/agreement/dig_tidn/index.html"
            ],
            "keyword": [
                "bibliotek",
                "forskning",
                "historia",
                "kulturarv"
            ],
            "alternative": null
        },
        {
            "@context": "https://id.kb.se/context.jsonld",
            "@id": "https://data.kb.se/dataset/hwpjh1n7fnsskvfr",
            "@type": "Dataset",
            "title": [
                "Norrköpings Weko-tidningar 1758-1786"
            ],
            "description": [
                "Digitaliserad version av Kungliga bibliotekets bestånd av Norrköpings Weko-Tidningar 1758-1786. Digitaliseringen genomfördes 2017. Specifikationer: https://www.kb.se/namespace/digark/deliveryspecification/agreement/dig_tidn/index.html"
            ],
            "keyword": [
                "bibliotek",
                "forskning",
                "historia",
                "kulturarv"
            ],
            "alternative": [
                "Norrkjöpings wecko-tidningar",
                "Norrköpings wecko-tidningar",
                "Norrköpings weckotidningar"
            ]
        },
        {
            "@context": "https://id.kb.se/context.jsonld",
            "@id": "https://data.kb.se/dataset/hwpjh11gfnk12r2z",
            "@type": "Dataset",
            "title": [
                "Svenska dagbladet"
            ],
            "description": [
                "Digitaliserad version av Svenska Dagbladet. Digitaliseringen är gjord från KB:s pappersförlagor och genomfördes 2013-2014. Specifikationer: https://www.kb.se/namespace/digark/deliveryspecification/agreement/dig_tidn/index.html"
            ],
            "keyword": [
                "bibliotek",
                "forskning",
                "historia",
                "kulturarv"
            ],
            "alternative": null
        },
        {
            "@context": "https://id.kb.se/context.jsonld",
            "@id": "https://data.kb.se/dataset/l0sqlrn8jcp5svln",
            "@type": "Dataset",
            "title": [
                "Post och inrikes tidningar 1645-1705"
            ],
            "description": [
                "Digitaliserad version av Kungliga bibliotekets bestånd av Post- och inrikes tidningar från 1645-1705. Digitaliseringen genomfördes 2017. Specifikationer: https://www.kb.se/namespace/digark/deliveryspecification/agreement/dig_tidn/index.html"
            ],
            "keyword": [
                "bibliotek",
                "forskning",
                "historia",
                "kulturarv"
            ],
            "alternative": [
                "Posttidningar",
                "Inrikes tidningar"
            ]
        },
        {
            "@context": "https://id.kb.se/context.jsonld",
            "@id": "https://data.kb.se/dataset/crkhjdgx9004lsld",
            "@type": "Dataset",
            "title": [
                "Lånestatistik från Kungliga biblioteket 1999-2015"
            ],
            "description": [
                "Data om lånebeställningar ur KB:s system Regina åren 1999-2015. Data för åren 1999-2000 är inkompletta. Fjärrlån ingår inte. Varje lån redovisas med information om lokalt id-nummer, librisnummer, tidpunkt och vissa grunddata om objektet."
            ],
            "keyword": [
                "bibliotek",
                "forskning",
                "historia",
                "kulturarv"
            ],
            "alternative": null
        },
        {
            "@context": "https://id.kb.se/context.jsonld",
            "@id": "https://data.kb.se/dataset/r5zw3frhp30w5zpx",
            "@type": "Dataset",
            "title": [
                "Svensk biblioteksstatistik 2010-"
            ],
            "description": [
                "Statistik för folkbibliotek, forskningsbibliotek, sjukhusbibliotek och skolbibliotek från år 2010 och framåt. Biblioteksstatistiken är en del av Sveriges officiella statistik som KB gör på uppdrag av regeringen."
            ],
            "keyword": [
                "bibliotek",
                "forskning",
                "historia",
                "kulturarv"
            ],
            "alternative": null
        },
        {
            "@context": "https://id.kb.se/context.jsonld",
            "@id": "https://data.kb.se/dataset/bqkdn5fv8t2zn578",
            "@type": "Dataset",
            "title": [
                "Libris auktoritetsdata"
            ],
            "description": [
                "Auktoritetsdata från Libris. Information om ämnesord, personer och organisationer. Feeds för att möjliggöra syndikering till lokala kopior och följa förändringar."
            ],
            "keyword": [
                "bibliotek",
                "forskning",
                "historia",
                "kulturarv"
            ],
            "alternative": null
        },
        {
            "@context": "https://id.kb.se/context.jsonld",
            "@id": "https://data.kb.se/dataset/9pjcnd5h7kgn0mww",
            "@type": "Dataset",
            "title": [
                "Svenska nationalbibliografin"
            ],
            "description": [
                "Nationalbibliografin är en heltäckande översikt över den svenska utgivningen och utgör en metadatakälla för bibliotek och andra användare. Här finns metadata om författare, utgivare och ämnen som beskriver den svenska utgivningen. Finns även tillgänglig via olika API:er."
            ],
            "keyword": [
                "bibliotek",
                "forskning",
                "historia",
                "kulturarv"
            ],
            "alternative": null
        },
        {
            "@context": "https://id.kb.se/context.jsonld",
            "@id": "https://data.kb.se/dataset/bqkzx5mq8qbflctz",
            "@type": "Dataset",
            "title": [
                "Ljudinspelningar från omkring 1900"
            ],
            "description": [
                "Digitaliserade fonografinspelningar ur Kungliga bibliotekets samlingar. Musik och tal från sekelskiftet 1800/1900"
            ],
            "keyword": [
                "bibliotek",
                "forskning",
                "historia",
                "kulturarv"
            ],
            "alternative": null
        },
        {
            "@context": "https://id.kb.se/context.jsonld",
            "@id": "https://data.kb.se/dataset/xbcmh87lvlb07fmw",
            "@type": "Dataset",
            "title": [
                "Ljudinspelningar från Brittiska östafrika (Kenya) 1911"
            ],
            "description": [
                "Etnografiska inspelningar gjorda 1911 av Gerhard Lindblom i Brittiska Ostafrika (idag Kenya)"
            ],
            "keyword": [
                "bibliotek",
                "forskning",
                "historia",
                "kulturarv"
            ],
            "alternative": null
        },
        {
            "@context": "https://id.kb.se/context.jsonld",
            "@id": "https://data.kb.se/dataset/r579793bp8stvlg8",
            "@type": "Dataset",
            "title": [
                "Kortkatalog över KB:s handskriftssamling"
            ],
            "description": [
                "Beskrivning av dataset saknas"
            ],
            "keyword": [
                "bibliotek",
                "forskning",
                "historia",
                "kulturarv"
            ],
            "alternative": null
        },
        {
            "@context": "https://id.kb.se/context.jsonld",
            "@id": "https://data.kb.se/dataset/8nqsrrqn6c52dmzk",
            "@type": "Dataset",
            "title": [
                "Utgifter för vetenskaplig publicering på svenska lärosäten 2017-"
            ],
            "description": [
                "Kostnadsdata för vad svenska lärosäten betalar för vetenskaplig publicering. Här ingår kostnader för prenumerationer, publicering och transformativa avtal (avtal som avser att sammanföra kostnader för att både läsa och publicera). Kostnaderna anges i svenska kronor (SEK), och är uppdelade på lärosäte och på förlag. Data är insamlade via Bibsamkonsortiet och enskilda lärosäten. Observera att data inte är fullständigt, KB har sedan 2018 ett regeringsuppdrag att samla in dessa kostnader och avser att successivt förbättra täckningsgraden i framtida arbete."
            ],
            "keyword": [
                "bibliotek",
                "forskning",
                "historia",
                "kulturarv"
            ],
            "alternative": null
        },
        {
            "@context": "https://id.kb.se/context.jsonld",
            "@id": "https://data.kb.se/dataset/r5798fsmpzs3qdwl",
            "@type": "Dataset",
            "title": [
                "Filmgranskningskort från Statens biografbyrå (censurkort) 1911-1921"
            ],
            "description": [
                "Filmgranskningskort (censurkort) inklusive diarium från Statens biografbyrå för perioden 1911-1921"
            ],
            "keyword": [
                "bibliotek",
                "forskning",
                "historia",
                "kulturarv"
            ],
            "alternative": null
        }
    ],
    "aggs": null,
    "search": {
        "mapping": [
            {
                "value": "*",
                "variable": "q"
            },
            {
                "value": "dataset",
                "variable": "@type",
                "up": {
                    "@id": "https://data.kb.se/search?q=%2A&genreForm=Tidningsnummer&limit=20"
                }
            }
        ]
    }
}